<a href="https://colab.research.google.com/github/datasigntist/tensorflowCodes/blob/master/tensorflow101_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import functools 
import numpy as np 
import tensorflow as tf 
import pandas as pd

LABEL_COLUMN = "Survived"

TRAIN_DATA_URL=  "https://raw.githubusercontent.com/datasigntist/datasetsForTraining/master/train.csv" 

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL) 

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

titanicDataSet = get_dataset(train_file_path)

class extractFeaturesFromName(object):
  #def __init__(self):
  #  print("Initiated")

  def __call__(self, features, labels):
    namesOfPassengers = features['Name']
    lastName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(feat,",")[0]) ,namesOfPassengers)
    firstName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[1]) ,namesOfPassengers)
    title_feature  = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[0]) ,namesOfPassengers)
    features['last_name'] = lastName_feature
    features['first_name'] = firstName_feature
    features['title_feature'] = title_feature
    return features, labels

class replaceMissingValuesInAge(object):
  def __init__(self, meanAge):
    self.meanAge = meanAge

  def __call__(self, features, labels):
    ageOfPassengers = features['Age']
    tf.print(ageOfPassengers)
    ageOfPassengers = tf.map_fn(lambda age: (age if age>0 else self.meanAge) ,ageOfPassengers)
    features['Age'] = ageOfPassengers
    return features, labels

desc = pd.read_csv(train_file_path)["Age"].describe()
MEAN = np.float32(np.array(desc.T['mean']))

newtitanicDataSet = titanicDataSet.map(extractFeaturesFromName())
newtitanicDataSet = titanicDataSet.map(replaceMissingValuesInAge(MEAN))

show_batch(newtitanicDataSet)

[0 41 0 0 0]
PassengerId         : [498 762 869 668  33]
Pclass              : [3 3 3 3 3]
Name                : [b'Shellard, Mr. Frederick William' b'Nirva, Mr. Iisakki Antino Aijo'
 b'van Melkebeke, Mr. Philemon' b'Rommetvedt, Mr. Knud Paust'
 b'Glynn, Miss. Mary Agatha']
Sex                 : [b'male' b'male' b'male' b'male' b'female']
Age                 : [29.699118 41.       29.699118 29.699118 29.699118]
SibSp               : [0 0 0 0 0]
Parch               : [0 0 0 0 0]
Ticket              : [b'C.A. 6212' b'SOTON/O2 3101272' b'345777' b'312993' b'335677']
Fare                : [15.1    7.125  9.5    7.775  7.75 ]
Cabin               : [b'' b'' b'' b'' b'']
Embarked            : [b'S' b'S' b'S' b'S' b'Q']
